# Student Loan Risk with Deep Learning

In [23]:
!pip install --upgrade tensorflow


In [24]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [25]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [26]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [27]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [28]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [29]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
loans_df.head

<bound method NDFrame.head of       payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0                 7.4               0.700               0.00          1.9   
1                 7.8               0.880               0.00          2.6   
2                 7.8               0.760               0.04          2.3   
3                11.2               0.280               0.56          1.9   
4                 7.4               0.700               0.00          1.9   
...               ...                 ...                ...          ...   
1594              6.2               0.600               0.08          2.0   
1595              5.9               0.550               0.10          2.2   
1596              6.3               0.510               0.13          2.3   
1597              5.9               0.645               0.12          2.0   
1598              6.0               0.310               0.47          3.6   

      alumni_success  study_major_code  time_to_completion  \
0              0.076              11.0                34.0   
1              0.098              25.0                67.0   
2              0.092              15.0                54.0   
3              0.075              17.0                60.0   
4              0.076              11.0                34.0   
...              ...               ...                 ...   
1594           0.090              32.0                44.0   
1595           0.062              39.0                51.0   
1596           0.076              29.0                40.0   
1597           0.075              32.0                44.0   
1598           0.067              18.0                42.0   

      finance_workshop_score  cohort_ranking  total_loan_score  \
0                    0.99780            3.51              0.56   
1                    0.99680            3.20              0.68   
2                    0.99700            3.26              0.65   
3                    0.99800            3.16              0.58   
4                    0.99780            3.51              0.56   
...                      ...             ...               ...   
1594                 0.99490            3.45              0.58   
1595                 0.99512            3.52              0.76   
1596                 0.99574            3.42              0.75   
1597                 0.99547            3.57              0.71   
1598                 0.99549            3.39              0.66   

      financial_aid_score  credit_ranking  
0                     9.4               0  
1                     9.8               0  
2                     9.8               0  
3                     9.8               1  
4                     9.4               0  
...                   ...             ...  
1594                 10.5               0  
1595                 11.2               1  
1596                 11.0               1  
1597                 10.2               0  
1598                 11.0               1  

[1599 rows x 12 columns]>

### Step 3: Split the features and target sets into training and testing datasets.


In [30]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)




### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [31]:
loan_scaled = StandardScaler()

# Fit the scaler to the features training dataset
loan_scaled.fit(X_train)


# Fit the scaler to the features training dataset
X_train_scaled = loan_scaled.transform(X_train)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [38]:
# Define the the number of inputs (features) to the model
num_input_features = X_train.shape[1]
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(num_input_features,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')])

# Review the number of features
print("Number of input features:", num_input_features)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Number of input features: 11


In [ ]:
# Define the number of hidden nodes for the first hidden layer


# Define the number of hidden nodes for the second hidden layer


# Define the number of neurons in the output layer


In [39]:
# Create the Sequential model instance
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(num_input_features,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')])

# Add the first hidden layer


# Add the second hidden layer


# Add the output layer to the model specifying the number of output neurons and activation function


In [40]:
# Display the Sequential model summary
print("Number of input features:", num_input_features)

Number of input features: 11


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [41]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [42]:
# Fit the model using 50 epochs and the training data
model.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6824 - loss: 0.6146
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7246 - loss: 0.5178
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7591 - loss: 0.5045
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7565 - loss: 0.4838
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7763 - loss: 0.4688
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7652 - loss: 0.4796
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7962 - loss: 0.4552
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7915 - loss: 0.4524
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7813 - loss: 0.4594
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7736 - loss: 0.4777
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8008 - loss: 0.4353
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7761 - 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [44]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
X_test_scaled = loan_scaled.transform(X_test)
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7667 - loss: 0.5950  
Test Loss: 0.6238884925842285
Test Accuracy: 0.7425000071525574


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [45]:
# Set the model's file path
model.save("student_loans.keras")


# Export your model to a keras file


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [46]:
# Set the model's file path


# Load the model to a new object
loaded_model = keras.models.load_model("student_loans.keras")


### Step 2: Make predictions on the testing data and save the predictions to a

DataFrame.

In [48]:
# Make predictions with the test data
predictions = model.predict(X_test_scaled)
predictions_df = pd.DataFrame(predictions, columns=['predicted_probability'])


# Display a sample of the predictions
print(predictions_df.head())

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
   predicted_probability
0               0.077524
1               0.318037
2               0.873062
3               0.586552
4               0.979055


In [50]:
# Save the predictions to a DataFrame and round the predictions to binary results
binary_predictions = (predictions > 0.5).astype(int)
predictions_df = pd.DataFrame(binary_predictions, columns=['prediction'])


### Step 4: Display a classification report with the y test data and predictions

In [51]:
# Print the classification report with the y test data and predictions
print(predictions_df.head())

   prediction
0           0
1           0
2           1
3           1
4           1


---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

